# 第1章 Python数据模型
## 1 一摞Python风格的纸牌

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
deck = FrenchDeck()
len(deck)

52

In [4]:
from random import choice
choice(deck)

Card(rank='Q', suit='spades')

查看最上面3张和只看牌面是A的牌：

In [5]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [6]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

仅仅实现了`__getitem__`方法，这一摞牌就变成可迭代的了：

In [7]:
for i, card in enumerate(reversed(deck)):
    if i <10 or i > 40:
        print(card)
    if i == 11:
        print('......\n......')

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
......
......
Card(rank='Q', suit='spades')
Card(rank='J', suit='spades')
Card(rank='10', suit='spades')
Card(rank='9', suit='spades')
Card(rank='8', suit='spades')
Card(rank='7', suit='spades')
Card(rank='6', suit='spades')
Card(rank='5', suit='spades')
Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


升序排名：

In [8]:
def spades_high(card):
    """
    1. 点数判定：2 最小、A最大
    2. 花色判定：黑桃最大、红桃次之、方块再次、梅花最小
    """
    rank_value = FrenchDeck.ranks.index(card.rank)
    suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [9]:
i = 0
for card in sorted(deck, key=spades_high):
    if i <10 or i > 47:
        print(card)
    if i == 11:
        print('......\n......')
    i += 1

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
......
......
Card(rank='A', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='A', suit='hearts')
Card(rank='A', suit='spades')


## 2 如何使用特殊方法
### 2.1 模拟数值类型

![image](http://static.zybuluo.com/AustinMxnet/iyahey7fown74bnzvbom0o6r/image.png)

实现一个二维向量（vector）类：

> Python内置的`complex`类可以用来表示二维向量，但我们这个自定义的类可以扩展到$n$维向量，详见第14章。

In [10]:
from math import hypot


class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        # return bool(self.x or self.y) # 更高效
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

加法和模运算：

In [11]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v1 + v2

Vector(4, 5)

In [12]:
v = Vector(3, 4)
abs(v)

5.0

### 2.2 字符串表现形式

在`__repr__`的实现中，**我们用到了`%r`来获取对象各个属性的标准字符串表示形式**---这是个好习惯，它暗示了一个关键：`Vector(1, 2)`和`Vector('1', '2')`是不一样的，后者在我们的定义中会报错，因为向量对象的构造函数只接受数值，不接受字符串。

In [13]:
v1 = Vector(2, 4)
v2 = Vector('2', '4')

print(v1, v2, sep='\n')

Vector(2, 4)
Vector('2', '4')
